In [18]:
# These are all the dependencies
# langchain lanchain-openai: for LangChain stuff
# request: for REST API calls
# mecab-python3 unidic-lite: for dividing Japanese sentences into words
# genanki: for creating Anki cards
! pip install -q langchain langchain-openai requests mecab-python3 unidic-lite genanki openai

In [19]:
# All the imports go here
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from google.colab import userdata
import requests
import MeCab
import urllib.parse
import genanki
import pprint
from openai import OpenAI as OpenAIClient

In [20]:
# Get a Japanese sentence from Tatoeba

SENTENCE_ID = 4704
TATOEBA_API_URL = "https://api.dev.tatoeba.org/unstable"

response = requests.get(f"{TATOEBA_API_URL}/sentences/{SENTENCE_ID}")

if response.status_code == 200:
    data = response.json()
else:
    print(f"Request failed with status code {response.status_code}")


In [21]:
# Inspect the answer from Tatoeba

print(f"Text: {data['data']['text']}")
print(f"Language: {data['data']['lang']}")

# There are 2 items in the translations array, why?
translations = [translation for translation in data['data']['translations'][0] if translation['lang'] == 'eng']
print(f"Available translations: {len(translations)}")
print(f"Translations: {translations[0]['text']}")

# This one has an audio file, download it
audio_file = requests.get(data['data']['audios'][0]['download_url'], allow_redirects=True)
open('audio.mp3', 'wb').write(audio_file.content)


Text: 何してるの？
Language: jpn
Available translations: 2
Translations: What are you doing?


10007

In [22]:
# Transcribe the Audio File with Whispers

client = OpenAIClient(api_key=userdata.get('OPENAI_API_KEY'))

audio_file= open("audio.mp3", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)
print(transcription.text)

何してるの?


In [23]:
# Create an llm and prompt template

llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    max_retries=2,
    api_key=userdata.get('OPENAI_API_KEY'),
)

prompt =  PromptTemplate.from_template((
    "Translate the following sentece to English: {sentence}" "\n"
    "Translation = "
))

In [24]:
# Translate the sentence

llm.invoke(prompt.format(sentence=data['data']['text']))

'What are you doing?'

In [25]:
# Divide the sentence into words
wakati = MeCab.Tagger("-Owakati")
words = wakati.parse(data['data']['text']).split()
print(words)

['何', 'し', 'てる', 'の', '？']


In [26]:
# Get the JLPT Classification for the words
JLPT_API_URL = "https://jlpt-vocab-api.vercel.app"

jlpt_classifications = []

for word in words:
  response = requests.get(f"{JLPT_API_URL}/api/words?word={urllib.parse.quote(word)}")

  if response.status_code == 200:
      jlpt_classifications.append(response.json())
      print(jlpt_classifications[-1])
  else:
      print(f"Request failed with status code {response.status_code}")

{'total': 2, 'offset': 0, 'limit': 10, 'words': [{'word': '何', 'meaning': 'what', 'furigana': 'なん / なに', 'romaji': 'nan / nani', 'level': 5}, {'word': '何', 'meaning': 'what', 'furigana': 'なに', 'romaji': 'nani', 'level': 3}]}
{'total': 1, 'offset': 0, 'limit': 10, 'words': [{'word': 'し', 'meaning': '10^24 (kanji is JIS X 0212 kuten 4906); septillion (American); quadrillion (British)', 'furigana': '', 'romaji': 'shi', 'level': 1}]}
{'total': 0, 'offset': 0, 'limit': 10, 'words': []}
{'total': 0, 'offset': 0, 'limit': 10, 'words': []}
{'total': 0, 'offset': 0, 'limit': 10, 'words': []}


In [27]:
# Generate Anki cards for the words

my_model = genanki.Model(
  1607392319,
  'Simple Model',
  fields=[
    {'name': 'Question'},
    {'name': 'Answer'},
  ],
  templates=[
    {
      'name': 'Card 1',
      'qfmt': '{{Question}}',
      'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}',
    },
  ])

my_deck = genanki.Deck(
  2059400110,
  'Japanese Words'
)

for classification in jlpt_classifications:
  if classification['total'] > 0:

    my_note = genanki.Note(
      model=my_model,
      fields=[classification['words'][0]['word'], classification['words'][0]['meaning']]
    )

    pprint.pp(my_note)

    my_deck.add_note(my_note)

genanki.Package(my_deck).write_to_file('output.apkg')

Note(model=Model(model_id=1607392319, name='Simple Model', fields=[{'name': 'Question'}, {'name': 'Answer'}], templates=[{'name': 'Card 1', 'qfmt': '{{Question}}', 'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}'}], css='', model_type=0), fields=['何', 'what'], sort_field='何', tags=_TagList([]), guid='FAJ#SAa)+@')
Note(model=Model(model_id=1607392319, name='Simple Model', fields=[{'name': 'Question'}, {'name': 'Answer'}], templates=[{'name': 'Card 1', 'qfmt': '{{Question}}', 'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}'}], css='', model_type=0), fields=['し', '10^24 (kanji is JIS X 0212 kuten 4906); septillion (American); quadrillion (British)'], sort_field='し', tags=_TagList([]), guid='b2UTAu0K!?')
